In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Paths to images and masks
image_dir = '/kaggle/input/probe-d/png_images/IMAGES/'
mask_dir = '/kaggle/input/probe-d/png_masks/MASKS/'

# Get list of image files
images = sorted([f for f in os.listdir(image_dir) if f.endswith('.png')])
masks = sorted([f for f in os.listdir(mask_dir) if f.endswith('.png')])


# Split the data into training and testing (80%-20%)
train_images, test_images, train_masks, test_masks = train_test_split(
    images, masks, test_size=0.2, random_state=33
)

# Save split data into CSV (this will be useful for tracking splits)
train_df = pd.DataFrame({
    'image': train_images,
    'mask': train_masks
})

test_df = pd.DataFrame({
    'image': test_images,
    'mask': test_masks
})

# Save dataframes for later use
train_df.to_csv('/kaggle/working/train_split.csv', index=False)
test_df.to_csv('/kaggle/working/test_split.csv', index=False)

# Check the first few entries of the training set
train_df.head()


,image,mask
0,img_0867.png,seg_0867.png
1,img_0910.png,seg_0910.png
2,img_0641.png,seg_0641.png
3,img_0583.png,seg_0583.png
4,img_0517.png,seg_0517.png


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import cv2

def load_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)  # Use OpenCV to read images
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
    img = cv2.resize(img, (825, 550))  # Ensure consistent size
    return img

def load_mask(mask_path):
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
    mask = cv2.resize(mask, (825, 550))  # Resize to match image size
    return mask

# Create a dataset loader
def create_dataset(image_paths, mask_paths):
    image_data = [load_image(os.path.join(image_dir, image)) for image in image_paths]
    mask_data = [load_mask(os.path.join(mask_dir, mask)) for mask in mask_paths]
    return np.array(image_data), np.array(mask_data)

# Example usage:
train_images_data, train_masks_data = create_dataset(train_images, train_masks)
test_images_data, test_masks_data = create_dataset(test_images, test_masks)


In [4]:
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [5]:
from tensorflow.keras import layers, models

def create_model(input_shape):
    model = models.Sequential([
        layers.InputLayer(input_shape=input_shape),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D(),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.UpSampling2D(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.UpSampling2D(),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(59, (1, 1), activation='softmax', padding='same'),
        layers.ZeroPadding2D(padding=((1, 1), (1, 0)))   # Adjust padding to match target size (825x550)
    ])
    return model

# Example model creation
input_shape = (550, 825, 3)  # Height, Width, Channels (RGB)
model = create_model(input_shape)
initial_learning_rate = 0.001  # You can experiment with this value
optimizer = Adam(learning_rate=initial_learning_rate)

# Use ReduceLROnPlateau to reduce learning rate when validation loss plateaus
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# Compile the model
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Display model summary
model.summary()




/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 550, 825, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 275, 412, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 275, 412, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 137, 206, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 137, 206, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d (UpSampling2D)         │ (None, 274, 412, 128)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 274, 412, 64)        │          73,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d_1 (UpSampling2D)       │ (None, 548, 824, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 548, 824, 32)        │          18,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 548, 824, 59)        │           1,947 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ zero_padding2d (ZeroPadding2D)       │ (None, 550, 825, 59)        │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 187,451 (732.23 KB)

 Trainable params: 187,451 (732.23 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(train_images_data, train_masks_data, batch_size=4, epochs=10, validation_split=0.2, verbose=1)



Epoch 1/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 2479s 15s/step - accuracy: 0.7340 - loss: 2.2266 - val_accuracy: 0.7668 - val_loss: 1.2319
Epoch 2/10
 42/160 ━━━━━━━━━━━━━━━━━━━━ 28:09 14s/step - accuracy: 0.7669 - loss: 1.3145

In [12]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_images_data, test_masks_data, verbose=1)

# Print the test loss and accuracy
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')




7/7 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - accuracy: 0.7802 - loss: 1.0060 
Test Loss: 1.014285922050476
Test Accuracy: 0.7786061763763428


In [13]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import save_img

# Make predictions on the test data
predictions = model.predict(test_images_data)

# Post-process predictions: Convert one-hot predictions to class labels
predicted_masks = np.argmax(predictions, axis=-1)  # Convert the output to class labels

# Save the predicted masks as PNG files
for i, mask in enumerate(predicted_masks):
    # Save each mask with the corresponding image name (make sure the names match the input images)
    output_filename = f'pred_mask_{i+1}.png'  # Modify the filename as needed
    save_img(output_filename, mask)  # Save mask as PNG
    print(f'Saved mask {output_filename}')


1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 498ms/step

ResourceExhaustedError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/pyt

In [1]:
import tensorflow as tf
from tensorflow.keras.backend import clear_session

clear_session()  # Clear any cached memory from previous models
